# HiPAS GridLAB-D 
# Developer Training Session 8
## Data and Model Converters
### David P. Chassin 
### Summer 2022
### dchassin@stanford.edu

SLAC National Accelerator Laboratory

Operated by Stanford University for the US Department of Energy


# Fundamentals

**Converters are automatic on the command line:**

    $ gridlabd INPUT1.EXT1 INPUT2.EXT2 ... -o OUTPUT.EXT3 [OPTIONS ...]`

* Syntax used for both data and model files

**Two important characteristics of all data and model files**

* *Format* - the structure of the data (this is the extension)

* *Semantics* - the meaning of the data (this is not always obvious)

**Format determined by default using file extension**

* Automatic converters output to GLM by default

# Semantic hints      

**Semantic information must be provided**

    $ gridlabd -D csv_load_options=ami ami_data.csv

**Semantic hints applied as they appear in command line**

* Use `<ext>_load_options='-t <type>` for input
* Use `<ext>_save_options='-f <format>` for output

**Example**

    $ gridlabd -D csv_load_options='-t ceus' ceus.csv -D csv_load_options='-t table' objects.csv -D json_save_options='-f profile' -o output.json



# Converter options

**Subcommand syntax**

    $ gridlabd convert -i INPUT -o OUTPUT -f TYPE1 -t TYPE2 [OPTIONS ...]

**Converters usually support URLs for input**

    $ gridlabd convert http://example.com/data.csv -o model.glm -f csv-table -t glm-object

**All converters work as GLM macros**

    #convert INPUT -o OUTPUT -f EXT-TYPE -t EXT-TYPE [OPTIONS ...]

# Formats and Semantics

| Input format | Output format | Input semantics | Output Semantics | Remarks
| :-- | :-- | :-- | :-- | :--
| csv | glm | ceus | ceus | Load data from CEUS websites to `commercial.ceus` object
|     |     | rbsa | rbsa | Load data from CEUS websites to `residential.rbsa` object
|     |     | config | config | CSV table to GLM `#define` statements
|     |     | modify | modify | CSV table to GLM `modify` directives
|     |     | noaa-weather | weather | Convert NOAA LCD weather file to GLM weather object
|     |     | onpoint-weather | weather | Convert Onpoint Weather file to GLM weather object
|     |     | scada | ceus, rbsa | Convert CSV SCADA file to CEUS or RBSA objects
|     |     | visualcrossing-weather | weather | Convert VisualCrossing weather file to GLM weather object
| csv | kml | geodata | | Convert geodata to KML 
| json | csv | gridlabd | profile | Generate voltage profile in CSV format
|      | html | gridlabd |  | Generate a viewable HTML file
|      | png | gridlabd | oneline | Generate a oneline diagram
| mdb  | glm | cyme |  | Convert Cyme models to GLM
| zip  | glm |  |  | Extract GLM file from ZIP

# CEUS data
**CEUS loadshape data**

    Month,Daytype,Hour,Heating,Cooling,Ventilation,Water_Heating,Cooking,Refrigeration,Exterior_Lig ...
    1,WEEKDAY,0,0.7264,0.0000,0.1312,0.0383,0.0003,0.0940,0.2565,0.0768,0.0839,0.2104,0.0342,0.0000
    1,WEEKDAY,1,0.7447,0.0000,0.1324,0.0383,0.0003,0.0940,0.2565,0.0768,0.0839,0.2104,0.0342,0.0000
    1,WEEKDAY,2,0.7703,0.0000,0.1335,0.0383,0.0003,0.0940,0.2565,0.0773,0.0839,0.2104,0.0342,0.0000
    1,WEEKDAY,3,0.7809,0.0000,0.1345,0.0384,0.0003,0.0940,0.2565,0.0843,0.0842,0.2107,0.0352,0.0007

**Subcommand example**

    $gridlabd convert -i FCZ01_AOFF.csv -o output_ceus.glm -f csv-ceus -t glm-ceus -p floor_area=25000sf -p weather=CA-Oakland_Metropolitan_Arpt.tmy3

**Output GLM**

    // generated by csv-ceus2glm-ceus.py
    object ceus {
        floor_area "25000sf";
        filename "FCZ01_AOFF.csv";
        weather "CA-Oakland_Metropolitan_Arpt.tmy3";
    }


#  RBSA data
**Input loadshape data**

    Month,Daytype,Hour,Appliances,Cooking,Cooling,Electronics,ExteriorLighting, ...
    1,WEEKDAY,0,0.1049,0.0054,0,0.0761,0,0.5003,1.4327,0.0446,0.0798,0.2213,0.1 ...
    1,WEEKDAY,1,0.1047,0.0069,0,0.0721,0,0.5941,1.5816,0.0440,0.0870,0.2243,0.1 ...

**Subcommand**

    $gridlabd convert -i input_rbsa.csv -o output_rbsa.glm -f csv-rbsa -t glm-rbsa -p 'floor_area=1500sf' -p composition='Heating:{ZR:0.9;PR:0.1;PI:0.01}' ...'

**Output GLM**

    object rbsa 
    {
        filename "input_rbsa.csv";
        floor_area 1500 sf;
        composition "Heating:{ZR:0.9;PR:0.1;PI:0.01}";
        ...
    }

# Globals table

**Input file**

    MY_INT,1
    MY_FLOAT,1.2
    MY_STRING,test string

**GLM file**

    #convert -i input.csv -o output.glm -f csv-config -t glm-config
    #include "output.glm"

**Output file**

    #define MY_INT=1
    #define MY_FLOAT=1.2
    #define MY_STRING=test string

# Property table

**Input file**

    load1.phases,ABC
    load1.constant_power_A,10 kW
    load1.constant_power_B,8 kW

**GLM file**

    module powerflow;
    object load {
        name load1;
        nominal_voltage 120 V;
    }
    #convert -i test_modify.csv -o test_modify_opt.glm -f csv-modify -t glm-modify
    #include "test_modify_opt.glm"

**Output file**

    modify load1.phases "ABC";
    modify load1.constant_power_A "10 kW";
    modify load1.constant_power_B "8 kW";

# Library data

**Input file**

    class,name,value1,value2
    test,item1,1.2,3.4
    test,item2,5.6,7.8

**GLM file**

    class test { double value1; double value2; }
    #convert -i ${DIR:-.}/test_library.csv -o test_library_opt.glm -f csv-table -t glm-library
    #include "test_library_opt.glm"

**Output file**

    object test
    {
        name "item1";
        value1 "1.2";
        value2 "3.4";
    } ...

# Object data

**Input file**

    name,value1,value2
    item1,1.2,3.4
    item2,5.6,7.8

**GLM file**

    class test { double value1; double value2; }
    #convert -i ${DIR:-.}/test_object.csv -o test_object_opt.glm -f csv-table -t glm-object class=test
    #include "test_object_opt.glm"

**Output file**

    object test { 
        name "item1";
        value1 1.2;
        value2 3.4;
    } ...

# NOAA weather

**Input file**

    "STATION","DATE","REPORT_TYPE","SOURCE","AWND","BackupDirection","BackupDistance ...
    "72594524283","2019-01-01T00:53:00","FM-15","7",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, ...
    "72594524283","2019-01-01T01:53:00","FM-15","7",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, ...
    "72594524283","2019-01-01T02:53:00","FM-15","7",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, ...
    "72594524283","2019-01-01T03:53:00","FM-15","7",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, ...

**GLM file**

    #input "https://s3-us-west-1.amazonaws.com/weather.gridlabd.us/test_data/noaa.csv" -o test_data.glm -f noaa-weather -t weather -p station_id=72594524283 -p latitude=37.7749 -p longitude=-122.4194 -p timezone=-07:00 

**Output GLM**

    module tape;
    class weather {
        char32 station_id;
        double temperature[degF];
        double humidity[%];
        ...

    ...
    object weather
    {
        name "test_data";
        station_id "72594524283";
        object player
        {
            file "test_data.csv";
            property "temperature[degF],humidity[%],wind_speed[mph],wind_dir[deg],pressure[inHg],rainfall[in/h],opq_sky_cov[pu],solar_global[W/sf],solar_direct[W/sf],solar_diffuse[W/sf]";
        };
    }

**Weather data**

    #datetime,temperature[degF],humidity[%],wind_speed[mph],wind_dir[deg],pressure[inHg],rainfall[in/h],opq_sky_cov[pu],solar_global[W/sf],solar_direct[W/sf],solar_diffuse[W/sf]
    2019-01-01 00:53:00,41.0,62.0,7.0,60.0,30.03,0.0,0.0,0.0,0.0,0.0
    2019-01-01 01:53:00,44.0,45.0,6.0,60.0,30.04,0.0,0.0,0.0,0.0,0.0
    2019-01-01 02:53:00,37.0,67.0,6.0,90.0,30.03,0.0,0.0,0.0,0.0,0.0
    2019-01-01 03:53:00,38.0,60.0,6.0,90.0,30.02,0.0,0.0,0.0,0.0,0.0

# TMY3 weather

**Input file**

    Date (MM/DD/YYYY),Time (HH:MM),ETR (W/m^2),ETRN (W/m^2),GHI (W/m^2),GHI source,G ...
    01/01/1998,01:00,0,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,1,E,9,1,E,9,5.7,E ...
    01/01/1998,02:00,0,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,2,E,9,2,E,9,5.8,E ...
    01/01/1998,03:00,0,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,0,2,0,2,E,9,2,E,9,5.9,E ...

**GLM file**

    clock {
        timezone "PST+8PDT";
        starttime "2022-01-01 00:00:00 PST";
        stoptime "2022-01-02 00:00:00 PST";
    }
    #weather get CA-Twentynine_Palms.tmy3
    #input "CA-Twentynine_Palms.tmy3"
    ...


    ...
    module tape { csv_header_type NAME; }
    object recorder {
        parent ${FIND class=climate};
        file test_weather.csv;
        interval -1;
        property temperature;
    }

**Output file**

    timestamp,temperature
    2022-01-01 00:00:00 PST,+44.422
    2022-01-01 01:00:00 PST,+42.262
    2022-01-01 02:00:00 PST,+42.442
    2022-01-01 03:00:00 PST,+42.622


# Cyme models

**Input file**

**Subcommand**

**Output file**

# TODO

**Input file**

**Subcommand**

**Output file**

# Using OpenFIDO

# Questions